In [66]:
import os
import cv2
import json
import numpy as np

In [74]:
json_filename = '99_Taganrog-Day-Night.json'
with open(json_filename) as json_file:
    json_data = json.load(json_file)
    json_file.close()

In [75]:
output_dir = 'datasets/Cropped_traffic_lights_TAGANROG'
if not(os.path.isdir(output_dir)):
    os.mkdir(output_dir)

In [76]:
for i, record in enumerate(json_data['annotations']):
    if record['conf'] == 'NAR':
        record['conf'] = 'R'
        json_data['annotations'][i] = record
    elif record['conf'] == 'RNA':
        record['conf'] = 'R'
        json_data['annotations'][i] = record
    elif record['conf'] == 'GY':
        record['conf'] = 'Y'
        json_data['annotations'][i] = record
    elif record['conf'] == 'RN':
        record['conf'] = 'R'
        json_data['annotations'][i] = record

In [77]:
for record_image in json_data['images']:
    file_name = record_image['file_name']
    image = cv2.imread('datasets/'+file_name)
    file_name = file_name.split('/')[-1]
    list_of_bboxes = [record_annot['bbox'] for record_annot in json_data['annotations'] 
                      if record_annot['image_id'] == record_image['id']]
    list_of_confs = [record_annot['conf'] for record_annot in json_data['annotations'] 
                      if record_annot['image_id'] == record_image['id']]
    if list_of_bboxes != []:
        for i,box in enumerate(list_of_bboxes):
            x1, y1, width, height = box[0], box[1], box[2], box[3]
            x2 = x1 + width
            y2 = y1 + height
            #x_left, x_right, y_top, y_bottom = width / 2, width/2, height / 2, height / 2
            x_left, x_right, y_top, y_bottom = 0,0,0,0
            x1, x2, y1, y2 = max(x1-x_left,0), min(x2+x_right,image.shape[1]), max(y1-y_top,0), min(y2+y_bottom,image.shape[0]) 
            x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)
            cropped_image = image[y1:y2+1, x1:x2+1]
            intermediate_dir = list_of_confs[i]
            if json_filename == '97_NAMI-Polygon-Day-Night_25_11_2019.json':
                day_night = record_image['file_name'].split('/')[-2]
            else:
                day_night = ''
            if not(os.path.isdir(output_dir+'/'+intermediate_dir)):
                os.mkdir(output_dir+'/'+intermediate_dir)
            cv2.imwrite(output_dir+'/'+intermediate_dir+'/'+day_night+'_'+file_name[:file_name.rfind('.')]+'_'+str(i)+file_name[file_name.rfind('.'):],cropped_image)